In [0]:
sc.version

Out[95]: '3.1.0'

In [0]:
#install required python libraries and load them
#%run ./install_python_package
%pip install bs4
%pip install requests
%pip install wget
%pip install pyspark

Python interpreter will be restarted.
Collecting bs4
 Downloading bs4-0.0.1.tar.gz (1.1 kB)
Collecting beautifulsoup4
 Downloading beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
Collecting soupsieve>1.2; python_version >= "3.0"
 Downloading soupsieve-2.2.1-py3-none-any.whl (33 kB)
Building wheels for collected packages: bs4
 Building wheel for bs4 (setup.py): started
 Building wheel for bs4 (setup.py): finished with status 'done'
 Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=4b26f907e130d108c445f0db1d6e03dea6a4d8749afcef37a2da5cdcc08e86fe
 Stored in directory: /root/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Installing collected packages: soupsieve, beautifulsoup4, bs4
Successfully installed beautifulsoup4-4.9.3 bs4-0.0.1 soupsieve-2.2.1
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e603436b-2570-4b18-8aea-936b8a6de9ea/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (2.24.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests) (2020.12.5)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests) (2.10)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests) (3.0.4)
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e603436b-2570-4b18-8aea-936b8a6de9ea/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting wget
 Downloading wget-3.2.zip (10 kB)
Building wheels for collected packages: wget
 Building wheel for wget (setup.py): started
 Building wheel for wget (setup.py): finished with status 'done'
 Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=95407e586a3933a23cbba941adb93b00c32ecb8addb69bf8b70c53c82bde6e93
 Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Installing collected packages: wget
Successfully installed wget-3.2
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e603436b-2570-4b18-8aea-936b8a6de9ea/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting pyspark
 Downloading pyspark-3.1.1.tar.gz (212.3 MB)
Collecting py4j==0.10.9
 Downloading py4j-0.10.9-py2.py3-none-any.whl (198 kB)
Building wheels for collected packages: pyspark
 Building wheel for pyspark (setup.py): started
 Building wheel for pyspark (setup.py): finished with status 'done'
 Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767607 sha256=a4d8237a05a35200ffdaa5cba29512ef018f701ad0b2b5ed84c08510fbf55b1e
 Stored in directory: /root/.cache/pip/wheels/b3/0e/81/264aeed961e43b9f6ba9ec81c8c540d2d7dccc52c6b51cbf22
Successfully built pyspark
Installing collected packages: py4j, pyspark
Successfully installed py4j-0.10.9 pyspark-3.1.1
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e603436b-2570-4b18-8aea-936b8a6de9ea/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import requests
import zipfile
import requests
from os.path import basename
import gzip
import shutil
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import dataframe


### Steps
  1) Create download folders in DRIVER (Common name for all coins) <br/>
  2) Creates coin specific folder structure in MOUNT  <br/>
  3) Gets the list of zip file download urls for coin type  <br/>
  4) Execute download (Downloaded to DRIVER)  <br/>
  5) Extract the zip files into TSV files (in DRIVER)  <br/>
  6) Move downloaded files, Extracts to coin specific folders in Mount  <br/>
  7) Load TSV files into dataframe  <br/>
  8) Save Selected columns from Dataframe into a CSV file  <br/>
  3) Copy CSV file to FILESTORE to download using Web Browser  <br/>

### Common Methods
These methods do not depend on coin specific folder structure, download things to DRIVER <br/>
1) Create download folders in Driver (Common name for all coins) <br/>
3) Gets the list of zip file download urls for coin type <br/>
4) Execute download (Downloaded to Driver) <br/>
5) Extract the zip files into TSV files (in Driver) <br/>

In [0]:
def create_driver_folders():
  dbutils.fs.mkdirs("file:/databricks/driver/ZipDownloads")
  dbutils.fs.mkdirs("file:/databricks/driver/ZipDownloads/Extracts")

In [0]:
def cleanup_driver_location():
  dbutils.fs.rm("file:/databricks/driver/ZipDownloads/",  True)
  dbutils.fs.rm("file:/databricks/driver/ZipDownloads/Extracts/",  True)

### (1) Address data

In [0]:
address_file_urls = [
["https://gz.blockchair.com/bitcoin/addresses/blockchair_bitcoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF", "BITCOIN"],
["https://gz.blockchair.com/bitcoin-cash/addresses/blockchair_bitcoin-cash_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF","BITCOIN-CASH"],
["https://gz.blockchair.com/bitcoin-sv/addresses/blockchair_bitcoin-sv_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF", "BITCOIN-SV"],
["https://gz.blockchair.com/dogecoin/addresses/blockchair_dogecoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF", "DOGECOIN"],
["https://gz.blockchair.com/dash/addresses/blockchair_dash_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF", "DASH"],
["https://gz.blockchair.com/litecoin/addresses/blockchair_litecoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF", "LITECOIN"]
]

Create download folders and Folders on Mount

In [0]:
def create_folder_structure():
  cleanup_driver_location()
  create_driver_folders() 
  dbutils.fs.rm("dbfs:/mnt/BlockChain/Address/TSVData", True)
  dbutils.fs.rm("dbfs:/mnt/BlockChain/Address/ZipFIles", True)
  dbutils.fs.mkdirs("dbfs:/mnt/BlockChain/Address/TSVData")
  dbutils.fs.mkdirs("dbfs:/mnt/BlockChain/Address/ZipFIles")

In [0]:
create_folder_structure()

Download the address Zip Files

Extract the donwloaded address Zip Files

In [0]:
%sh
wget "https://gz.blockchair.com/bitcoin/addresses/blockchair_bitcoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF"

--2021-04-16 00:08:27-- https://gz.blockchair.com/bitcoin/addresses/blockchair_bitcoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF
Resolving gz.blockchair.com (gz.blockchair.com)... 109.233.109.29
Connecting to gz.blockchair.com (gz.blockchair.com)|109.233.109.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1045441276 (997M) [application/octet-stream]
Saving to: ‘blockchair_bitcoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF’

 0K .......... .......... .......... .......... .......... 0% 191K 89m5s
 50K .......... .......... .......... .......... .......... 0% 191K 89m2s
 100K .......... .......... .......... .......... .......... 0% 191K 89m4s
 150K .......... .......... .......... .......... .......... 0% 191K 89m3s
 200K .......... .......... .......... .......... .......... 0% 32.6M 71m20s
 250K .......... .......... .......... .......... .......... 0% 193K 74m10s
 300K .......... .......... .......... .......... .......... 0% 18.2M 63m42s
 350K .......... .......... .......... .......... .......... 0% 95.9M 55m45s
 400K .......... .......... .......... .......... .......... 0% 136M 49m34s
 450K .......... .......... .......... .......... .......... 0% 120M 44m38s
 500K .......... .......... .......... .......... .......... 0% 194K 48m32s
 550K .......... .......... .......... .......... .......... 0% 20.0M 44m33s
 600K .......... .......... .......... .......... .......... 0% 21.1M 41m11s
 650K .......... .......... .......... .......... .......... 0% 40.7M 38m16s
 700K .......... .......... .......... .......... .......... 0% 41.4M 35m44s
 750K .......... .......... .......... .......... .......... 0% 66.4M 33m31s
 800K .......... .......... .......... .......... .......... 0% 92.2M 31m33s
 850K .......... .......... .......... .......... .......... 0% 96.5M 29m49s
 900K .......... .......... .......... .......... .......... 0% 99.5M 28m15s
 950K .......... .......... .......... .......... .......... 0% 89.5M 26m51s
 1000K .......... .......... .......... .......... .......... 0% 198K 29m39s
 1050K .......... .......... .......... .......... .......... 0% 60.0M 28m19s
 1100K .......... .......... .......... .......... .......... 0% 81.6M 27m5s
 1150K .......... .......... .......... .......... .......... 0% 63.4M 25m58s
 1200K .......... .......... .......... .......... .......... 0% 118M 24m56s
 1250K .......... .......... .......... .......... .......... 0% 64.1M 23m59s
 1300K .......... .......... .......... .......... .......... 0% 67.7M 23m6s
 1350K .......... .......... .......... .......... .......... 0% 26.8M 22m18s
 1400K .......... .......... .......... .......... .......... 0% 27.2M 21m33s
 1450K .......... .......... .......... .......... .......... 0% 27.8M 20m51s
 1500K .......... .......... .......... .......... .......... 0% 10.9M 20m14s
 1550K .......... .......... .......... .......... .......... 0% 31.2M 19m37s
 1600K .......... .......... .......... .......... .......... 0% 30.7M 19m2s
 1650K .......... .......... .......... .......... .......... 0% 34.1M 18m29s
 1700K .......... .......... .......... .......... .......... 0% 33.0M 17m58s
 1750K .......... .......... .......... .......... .......... 0% 27.3M 17m29s
 1800K .......... .......... .......... .......... .......... 0% 33.6M 17m2s
 1850K .......... .......... .......... .......... .......... 0% 31.5M 16m36s
 1900K .......... .......... .......... .......... .......... 0% 34.8M 16m11s
 1950K .......... .......... .......... .......... .......... 0% 28.0M 15m47s
 2000K .......... .......... .......... .......... .......... 0% 214K 17m20s
 2050K .......... .......... .......... .......... .......... 0% 6.53M 16m59s
 2100K .......... .......... .......... .......... .......... 0% 14.3M 16m37s
 2150K .......... .......... .......... .......... .......... 0% 38.3M 16m15s
 2200K .......... .......... .......... .......... .......... 0% 47.3M 15m54s
 2250K .......... .......... .......... .......... .......... 0% 209M 

In [0]:
%sh
wget "https://gz.blockchair.com/bitcoin-cash/addresses/blockchair_bitcoin-cash_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF"

--2021-04-16 00:13:05-- https://gz.blockchair.com/bitcoin-cash/addresses/blockchair_bitcoin-cash_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF
Resolving gz.blockchair.com (gz.blockchair.com)... 109.233.109.29
Connecting to gz.blockchair.com (gz.blockchair.com)|109.233.109.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 475688186 (454M) [application/octet-stream]
Saving to: ‘blockchair_bitcoin-cash_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF’

 0K .......... .......... .......... .......... .......... 0% 198K 39m4s
 50K .......... .......... .......... .......... .......... 0% 199K 39m2s
 100K .......... .......... .......... .......... .......... 0% 198K 39m1s
 150K .......... .......... .......... .......... .......... 0% 1.14M 30m55s
 200K .......... .......... .......... .......... .......... 0% 239K 31m12s
 250K .......... .......... .......... .......... .......... 0% 33.4M 26m2s
 300K .......... .......... .......... .......... .......... 0% 120M 22m19s
 350K .......... .......... .......... .......... .......... 0% 200K 24m21s
 400K .......... .......... .......... .......... .......... 0% 44.7M 21m40s
 450K .......... .......... .......... .......... .......... 0% 42.8M 19m31s
 500K .......... .......... .......... .......... .......... 0% 70.9M 17m45s
 550K .......... .......... .......... .......... .......... 0% 46.2M 16m17s
 600K .......... .......... .......... .......... .......... 0% 45.8M 15m2s
 650K .......... .......... .......... .......... .......... 0% 58.0M 13m58s
 700K .......... .......... .......... .......... .......... 0% 1.28M 13m26s
 750K .......... .......... .......... .......... .......... 0% 49.6M 12m36s
 800K .......... .......... .......... .......... .......... 0% 240K 13m45s
 850K .......... .......... .......... .......... .......... 0% 26.5M 13m0s
 900K .......... .......... .......... .......... .......... 0% 23.8M 12m20s
 950K .......... .......... .......... .......... .......... 0% 28.8M 11m44s
 1000K .......... .......... .......... .......... .......... 0% 30.5M 11m11s
 1050K .......... .......... .......... .......... .......... 0% 38.1M 10m41s
 1100K .......... .......... .......... .......... .......... 0% 49.7M 10m13s
 1150K .......... .......... .......... .......... .......... 0% 97.2M 9m48s
 1200K .......... .......... .......... .......... .......... 0% 201M 9m24s
 1250K .......... .......... .......... .......... .......... 0% 162M 9m3s
 1300K .......... .......... .......... .......... .......... 0% 224M 8m43s
 1350K .......... .......... .......... .......... .......... 0% 164M 8m24s
 1400K .......... .......... .......... .......... .......... 0% 1.56M 8m16s
 1450K .......... .......... .......... .......... .......... 0% 47.3M 8m0s
 1500K .......... .......... .......... .......... .......... 0% 57.5M 7m45s
 1550K .......... .......... .......... .......... .......... 0% 37.9M 7m31s
 1600K .......... .......... .......... .......... .......... 0% 243K 8m15s
 1650K .......... .......... .......... .......... .......... 0% 53.3M 8m0s
 1700K .......... .......... .......... .......... .......... 0% 56.0M 7m47s
 1750K .......... .......... .......... .......... .......... 0% 77.8M 7m34s
 1800K .......... .......... .......... .......... .......... 0% 102M 7m22s
 1850K .......... .......... .......... .......... .......... 0% 58.1M 7m10s
 1900K .......... .......... .......... .......... .......... 0% 103M 6m59s
 1950K .......... .......... .......... .......... .......... 0% 66.7M 6m49s
 2000K .......... .......... .......... .......... .......... 0% 95.6M 6m39s
 2050K .......... .......... .......... .......... .......... 0% 107M 6m30s
 2100K .......... .......... .......... .......... .......... 0% 101M 6m21s
 2150K .......... .......... .......... .......... .......... 0% 89.6M 6m12s
 2200K .......... .......... .......... .......... .......... 0% 124M 6m4s
 2250K .......... .......... .......... .......... .......... 0% 88.1M 5m56s
 2300K ..

In [0]:
%sh
wget "https://gz.blockchair.com/bitcoin-sv/addresses/blockchair_bitcoin-sv_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF"

--2021-04-16 00:41:00-- https://gz.blockchair.com/bitcoin-sv/addresses/blockchair_bitcoin-sv_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF
Resolving gz.blockchair.com (gz.blockchair.com)... 109.233.109.29
Connecting to gz.blockchair.com (gz.blockchair.com)|109.233.109.29|:443... connected.
HTTP request sent, awaiting response... 402 Payment Required
2021-04-16 00:41:01 ERROR 402: Payment Required.

In [0]:
%sh
wget "https://gz.blockchair.com/dogecoin/addresses/blockchair_dogecoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF"

--2021-04-16 00:42:52-- https://gz.blockchair.com/dogecoin/addresses/blockchair_dogecoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF
Resolving gz.blockchair.com (gz.blockchair.com)... 109.233.109.29
Connecting to gz.blockchair.com (gz.blockchair.com)|109.233.109.29|:443... connected.
HTTP request sent, awaiting response... 402 Payment Required
2021-04-16 00:42:53 ERROR 402: Payment Required.

In [0]:
%sh
wget "https://gz.blockchair.com/dash/addresses/blockchair_dash_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF"

In [0]:
%sh
wget "https://gz.blockchair.com/litecoin/addresses/blockchair_litecoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF"


In [0]:
%sh
ls -all  /databricks/driver

total 163308
drwxr-xr-x 1 root root 4096 Apr 16 02:59 .
drwxr-xr-x 1 root root 4096 Apr 16 02:32 ..
-rw-r--r-- 1 root root 4101448 Apr 16 02:15 .blockchair_bitcoin_sv_addresses_latest_tsv.gz.crc
-rw-r--r-- 1 root root 290020 Apr 16 02:18 .blockchair_dash_addresses_latest__1__tsv.gz.crc
-rw-r--r-- 1 root root 792900 Apr 16 02:19 .blockchair_dogecoin_addresses_latest_tsv.gz.crc
-rw-r--r-- 1 root root 678236 Apr 16 02:20 .blockchair_litecoin_addresses_latest_tsv.gz.crc
drwxr-xr-x 3 root root 4096 Apr 16 02:49 ZipDownloads
-rw-r--r-- 1 root root 161322312 Apr 16 02:19 blockchair_dogecoin_addresses_latest_tsv
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
drwxr-xr-x 3 root root 4096 Apr 16 00:04 eventlogs
drwxr-xr-x 2 root root 4096 Apr 16 03:00 ganglia
drwxr-xr-x 2 root root 4096 Apr 16 03:00 logs

In [0]:
dbutils.fs.mv("file:/databricks/driver/blockchair_bitcoin-cash_addresses_latest.tsv", "dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin-cash_addresses_latest.tsv")

Out[66]: True

In [0]:
dbutils.fs.mv("file:/databricks/driver/blockchair_bitcoin_addresses_latest.tsv", "dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin_addresses_latest.tsv")

Out[67]: True

In [0]:
dbutils.fs.mv("file:/databricks/driver/blockchair_bitcoin_sv_addresses_latest_tsv", "dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin_sv_addresses_latest_tsv")

Out[68]: True

In [0]:
dbutils.fs.mv("file:/databricks/driver/blockchair_dash_addresses_latest__1__tsv", "dbfs:/mnt/BlockChain/Address/TSVData/blockchair_dash_addresses_latest__1__tsv")

Out[69]: True

In [0]:
dbutils.fs.mv("file:/databricks/driver/blockchair_litecoin_addresses_latest_tsv", "dbfs:/mnt/BlockChain/Address/TSVData/blockchair_litecoin_addresses_latest_tsv")

Out[70]: True

In [0]:
dbutils.fs.mv("file:/databricks/driver/blockchair_dogecoin_addresses_latest_tsv", "dbfs:/mnt/BlockChain/Address/TSVData/blockchair_dogecoin_addresses_latest_tsv")

Out[74]: True

In [0]:
import os  
os.rename('/databricks/driver/blockchair_bitcoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF','/databricks/driver/blockchair_bitcoin_addresses_latest.tsv.gz')

In [0]:
dbutils.fs.cp("dbfs:/FileStore/tables/blockchair_bitcoin_sv_addresses_latest_tsv.gz", "file:/databricks/driver/")

Out[24]: True

In [0]:
dbutils.fs.cp("file:/databricks/driver/blockchair_bitcoin_addresses_latest.tsv.gz?key=202001ZjMvj8R3BF", "file:/databricks/blockchair_bitcoin_addresses_latest.tsv.gz")

Out[45]: True

In [0]:
dbutils.fs.mv("dbfs:/FileStore/tables/blockchair_dash_addresses_latest__1__tsv.gz", "file:/databricks/driver/blockchair_dash_addresses_latest__1__tsv.gz")

Out[25]: True

In [0]:
dbutils.fs.mv("dbfs:/FileStore/tables/blockchair_dogecoin_addresses_latest_tsv.gz", "file:/databricks/driver/blockchair_dogecoin_addresses_latest_tsv.gz")

Out[27]: True

In [0]:
dbutils.fs.mv("dbfs:/FileStore/tables/blockchair_litecoin_addresses_latest_tsv.gz", "file:/databricks/driver/blockchair_litecoin_abddresses_latest_tsv.gz")

Out[28]: True

In [0]:
%sh
gunzip /databricks/driver/blockchair_bitcoin_addresses_latest.tsv.gz

Load into a dataframes

In [0]:
dbutils.fs.ls("dbfs:/mnt/BlockChain/Address/TSVData")

Out[75]: [FileInfo(path='dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin-cash_addresses_latest.tsv', name='blockchair_bitcoin-cash_addresses_latest.tsv', size=801516522),
 FileInfo(path='dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin_addresses_latest.tsv', name='blockchair_bitcoin_addresses_latest.tsv', size=1612456213),
 FileInfo(path='dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin_sv_addresses_latest_tsv', name='blockchair_bitcoin_sv_addresses_latest_tsv', size=810443632),
 FileInfo(path='dbfs:/mnt/BlockChain/Address/TSVData/blockchair_dash_addresses_latest__1__tsv', name='blockchair_dash_addresses_latest__1__tsv', size=57551376),
 FileInfo(path='dbfs:/mnt/BlockChain/Address/TSVData/blockchair_dogecoin_addresses_latest_tsv', name='blockchair_dogecoin_addresses_latest_tsv', size=161322312),
 FileInfo(path='dbfs:/mnt/BlockChain/Address/TSVData/blockchair_litecoin_addresses_latest_tsv', name='blockchair_litecoin_addresses_latest_tsv', size=132284218)]

In [0]:
address_bitcoin_df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', delimiter='\t', inferschema='true').load('dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin_addresses_latest.tsv')

address_bitcoin_cash_df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', delimiter='\t', inferschema='true').load('dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin-cash_addresses_latest.tsv')

address_bitcoin_sv_df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', delimiter='\t', inferschema='true').load('dbfs:/mnt/BlockChain/Address/TSVData/blockchair_bitcoin_sv_addresses_latest_tsv')

address_dash_df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', delimiter='\t', inferschema='true').load('dbfs:/mnt/BlockChain/Address/TSVData/blockchair_dash_addresses_latest__1__tsv')

address_lite_coin_df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', delimiter='\t', inferschema='true').load('dbfs:/mnt/BlockChain/Address/TSVData/blockchair_litecoin_addresses_latest_tsv')

address_dogecoin_cash_df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', delimiter='\t', inferschema='true').load('dbfs:/mnt/BlockChain/Address/TSVData/blockchair_dogecoin_addresses_latest_tsv')

In [0]:
print (address_bitcoin_df.count() ,address_bitcoin_sv_df.count() ,address_dash_df.count() ,address_bitcoin_cash_df.count() , address_lite_coin_df.count(), address_dogecoin_cash_df.count())

38110846 19816022 1356729 16361526 3006529 3539913

In [0]:
address_bitcoin_cash_df.count()

Out[39]: 3022507

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import col

In [0]:
address_bitcoin_df = address_bitcoin_df.withColumnRenamed("_c0", "address").withColumnRenamed("_c1", "balance")
address_bitcoin_df = address_bitcoin_df.select(col("address"),col("balance"),lit("BitCoin").alias("Name"))

In [0]:
address_bitcoin_sv_df = address_bitcoin_sv_df.withColumnRenamed("_c0", "address").withColumnRenamed("_c1", "balance")
address_bitcoin_sv_df = address_bitcoin_sv_df.select(col("address"),col("balance"),lit("BitCoin-SV").alias("Name"))

In [0]:
address_dash_df = address_dash_df.withColumnRenamed("_c0", "address").withColumnRenamed("_c1", "balance")
address_dash_df = address_dash_df.select(col("address"),col("balance"),lit("Dash").alias("Name"))

In [0]:
address_lite_coin_df = address_lite_coin_df.withColumnRenamed("_c0", "address").withColumnRenamed("_c1", "balance")
address_lite_coin_df = address_lite_coin_df.select(col("address"),col("balance"),lit("LiteCoin-Cash").alias("Name"))

In [0]:
address_dogecoin_cash_df = address_dogecoin_cash_df.withColumnRenamed("_c0", "address").withColumnRenamed("_c1", "balance")
address_dogecoin_cash_df = address_dogecoin_cash_df.select(col("address"),col("balance"),lit("DogeCoin").alias("Name"))

In [0]:
address_bitcoin_cash_df = address_bitcoin_cash_df.withColumnRenamed("_c0", "address").withColumnRenamed("_c1", "balance")
address_bitcoin_cash_df = address_bitcoin_cash_df.select(col("address"),col("balance"),lit("BitCoin-Cash").alias("Name"))

In [0]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

In [0]:
new_merged_df_1 = address_bitcoin_df.union(address_bitcoin_sv_df)

In [0]:
new_merged_df_2 = address_dash_df.union(address_bitcoin_cash_df)

In [0]:
new_merged_df_3 = address_lite_coin_df.union(address_dogecoin_cash_df)

In [0]:
new_merged_df_2 = new_merged_df_2.union(new_merged_df_1)

In [0]:
new_merged_df_3 =new_merged_df_3.union(new_merged_df_2)

In [0]:
new_merged_df_3.count()

Out[93]: 82191565

In [0]:
print ( address_bitcoin_df.count() ,address_bitcoin_sv_df.count() ,address_dash_df.count() ,address_bitcoin_cash_df.count() , address_lite_coin_df.count(), address_dogecoin_cash_df.count())
(12) Spark Jobs
37978757 19825369 1357077 16358274 3022507 3551936

In [0]:
new_merged_df_3.select("Name").distinct().collect()

Out[95]: [Row(Name='LiteCoin-Cash'),
 Row(Name='DogeCoin'),
 Row(Name='Dash'),
 Row(Name='BitCoin-Cash'),
 Row(Name='BitCoin'),
 Row(Name='BitCoin-SV')]

In [0]:
new_merged_df_3[["address", "balance","Name"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Address/Merged_Address.csv")
 #dbutils.fs.cp("dbfs:/mnt/BlockChain/Blocks/Bitcoin-cash/bc_block_sub_df_exported.csv", "dbfs:/FileStore/bc_block_sub_df_exported.csv", recurse = True)

In [0]:
address_bitcoin_df[["address", "balance","Name"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Address/CSVData/bitcoin_address.csv")

In [0]:
address_bitcoin_cash_df[["address", "balance","Name"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Address/CSVData/bitcoin_cash_address.csv")

In [0]:
address_bitcoin_sv_df[["address", "balance","Name"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Address/CSVData/bitcoin_sv_address.csv")

In [0]:
address_dash_df[["address", "balance","Name"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Address/CSVData/dash_address.csv")

In [0]:
address_lite_coin_df[["address", "balance","Name"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Address/CSVData/litecoin_address.csv")

In [0]:
address_dogecoin_cash_df[["address", "balance","Name"]].coalesce(1).write.option("header", "true").csv("dbfs:/mnt/BlockChain/Address/CSVData/dogecoin_address_df.csv")

In [0]:
dbutils.fs.cp("dbfs:/mnt/BlockChain/Address/CSVData/bitcoin_address.csv", "dbfs:/FileStore/bitcoin_address.csv", recurse = True)
dbutils.fs.cp("dbfs:/mnt/BlockChain/Address/CSVData/bitcoin_cash_address.csv", "dbfs:/FileStore/bitcoin_cash_address.csv", recurse = True)
dbutils.fs.cp("dbfs:/mnt/BlockChain/Address/CSVData/bitcoin_sv_address.csv", "dbfs:/FileStore/bitcoin_sv_address.csv", recurse = True)
dbutils.fs.cp("dbfs:/mnt/BlockChain/Address/CSVData/dash_address.csv", "dbfs:/FileStore/dash_address.csv", recurse = True)
dbutils.fs.cp("dbfs:/mnt/BlockChain/Address/CSVData/litecoin_address.csv", "dbfs:/FileStore/litecoin_address.csv", recurse = True)
dbutils.fs.cp("dbfs:/mnt/BlockChain/Address/CSVData/dogecoin_address_df.csv", "dbfs:/FileStore/dogecoin_address_df.csv", recurse = True)

Out[25]: True

In [0]:
dbutils.fs.cp("dbfs:/mnt/BlockChain/Address/CSVData/dogecoin_address_df.csv", "dbfs:/FileStore/dogecoin_address_df.csv", recurse = True)

Out[26]: True